In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("results_full.csv")
df = df[df["split_method"] == "random_split"]

In [11]:
import pandas as pd

df = pd.read_csv("results_full.csv")
latex_table = df.to_latex(index=False,escape = True, float_format="%.2e")  # or adjust format
with open("results_table_IP.tex", "w") as f:
    f.write(latex_table)


In [12]:
rename_map = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMRegressor': 'GBT',
    'LGBMClassifier': 'GBT',
    'LinearRegressor': 'Lin. Regr.',
    'RandomForestRegressor': 'RF',
    'RandomForestClassifier': 'RF',
    'ResNet': 'ResNet',
    'TabPFNRegressor': 'TabPFN',
    'TabPFNClassifier': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_LogLoss': 'GPBoost',
    'GPBoost_Accuracy': 'GPBoost',
    'GPBoost_CRPS': 'GPBoost',
    'GPBoost_RMSE': 'GPBoost'

}


Average Difference and Average Relative Difference RMSE

In [13]:
metric = "RMSE"
dfm   = df[df["metric"] == metric]

pivot_rmse = dfm.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_rmse = pivot_rmse.rename(columns=rename_map)

pivot_rmse.to_csv(f"avg_{metric}_per_task_per_model_IP.csv", float_format="%.3f")


In [14]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_rmse = avg_relative_diff(pivot_rmse, error_metric=True)

print("Average relative difference (RMSE) - Interpolation :")
print(avg_diff_rmse.round(2))


Average relative difference (RMSE) - Interpolation :
model
Const.            358.24
Engression        461.82
FT-Trans.          33.54
GPBoost       2350328.03
GBT                43.05
Lin. Regr.      76934.00
MLP               116.86
RF                 50.21
ResNet            115.63
TabPFN             19.09
dtype: float64


In [15]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_rmse = normalized_accuracy(pivot_rmse, error_metric=True)
print("Average normalized accuracy (from RMSE) - Interpolation:")
print((100 * avg_norm_acc_rmse).round(2))


Average normalized accuracy (from RMSE) - Interpolation:
model
Const.         8.62
Engression    36.15
FT-Trans.     63.39
GPBoost       46.81
GBT           74.74
Lin. Regr.     7.79
MLP           50.69
RF            67.26
ResNet        39.49
TabPFN        90.46
dtype: float64


In [16]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_rmse = avg_rank(pivot_rmse, error_metric=True)
print("Average rank (RMSE) - Interpolation:")
print(avg_rank_rmse.round(2))



Average rank (RMSE) - Interpolation:
model
Const.        8.92
Engression    6.14
FT-Trans.     4.30
GPBoost       5.58
GBT           3.64
Lin. Regr.    8.28
MLP           5.11
RF            4.11
ResNet        6.14
TabPFN        1.94
dtype: float64


In [17]:
avg_diff    = avg_relative_diff(pivot_rmse, error_metric=True)           # in %
avg_acc     = normalized_accuracy(pivot_rmse, error_metric=True) * 100   # convert to % 
avg_rank    = avg_rank(pivot_rmse, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff, avg_acc, avg_rank],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries = pd.concat([pivot_rmse, metrics])

out = pivot_with_summaries.reset_index().rename(columns={"index":"task_id"})

out.to_csv(f"avg_{metric}_with_summary_IP.csv", float_format="%.3f", index=False)

latex = out.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric}_IP_results.tex","w") as f:
    f.write(latex)

Average Difference and Average Relative Difference CRPS

In [18]:
metric1 = "CRPS"
dfm1   = df[df["metric"] == metric1]

pivot_crps = dfm1.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_crps = pivot_crps.rename(columns=rename_map)
pivot_crps.to_csv(f"avg_{metric1}_per_task_per_model_IP.csv", float_format="%.3f")


In [19]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_crps = avg_relative_diff(pivot_crps, error_metric=True)

print("Average relative difference (CRPS) - Interpolation:")
print(avg_diff_crps.round(2))


Average relative difference (CRPS) - Interpolation:
model
Const.            578.10
DGBT               59.53
DRF                77.18
Engression        453.08
FT-Trans.          49.86
GPBoost       2467136.32
GBT                58.79
Lin. Regr.      54390.20
MLP                84.79
RF                 68.99
ResNet            127.33
TabPFN             22.61
dtype: float64


In [20]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_crps = normalized_accuracy(pivot_crps, error_metric=True)
print("Average normalized accuracy (from CRPS) - Interpolation:")
print((100 * avg_norm_acc_crps).round(2))


Average normalized accuracy (from CRPS) - Interpolation:
model
Const.         4.44
DGBT          68.51
DRF           70.95
Engression    21.39
FT-Trans.     59.16
GPBoost       47.69
GBT           68.47
Lin. Regr.     7.30
MLP           52.16
RF            60.23
ResNet        46.89
TabPFN        81.94
dtype: float64


In [21]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_crps = avg_rank(pivot_crps, error_metric=True)
print("Average rank (CRPS) - Interpolation:")
print(avg_rank_crps.round(2))



Average rank (CRPS) - Interpolation:
model
Const.        10.89
DGBT           4.28
DRF            4.86
Engression     9.25
FT-Trans.      5.24
GPBoost        7.03
GBT            4.33
Lin. Regr.     9.92
MLP            6.53
RF             5.44
ResNet         6.75
TabPFN         2.50
dtype: float64


In [28]:
avg_diff1    = avg_relative_diff(pivot_crps, error_metric=True)           # in %
avg_acc1     = normalized_accuracy(pivot_crps, error_metric=True) * 100   # convert to % 
avg_rank1    = avg_rank(pivot_crps, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff1, avg_acc1, avg_rank1],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries1 = pd.concat([pivot_crps, metrics])

out1 = pivot_with_summaries1.reset_index().rename(columns={"index":"task_id"})

out1.to_csv(f"avg_{metric1}_with_summary_IP.csv", float_format="%.3f", index=False)

latex1 = out1.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric1}_IP_results.tex","w") as f:
    f.write(latex1)

Average Difference and Average Relative Difference LOGLOSS

In [23]:
metric2 = "LogLoss"
dfm2   = df[df["metric"] == metric2]

pivot_ll = dfm2.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_ll = pivot_ll.rename(columns=rename_map)
pivot_ll.to_csv(f"avg_{metric2}_per_task_per_model_IP.csv", float_format="%.3f")


In [24]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    return rel_diff.mean(axis=0)


avg_diff_ll = avg_relative_diff(pivot_ll, error_metric=True)

print("Average relative difference (LogLoss) - Interpolation:")
print(avg_diff_ll.round(2))


Average relative difference (LogLoss) - Interpolation:
model
Const.        265.71
Engression    173.35
FT-Trans.      15.68
GPBoost        79.10
GBT           231.22
Log. Regr.    153.71
MLP            20.67
RF            231.22
ResNet         31.34
TabPFN          3.32
dtype: float64


In [25]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_ll = normalized_accuracy(pivot_ll, error_metric=True)
print("Average normalized accuracy (from LogLoss) - Interpolation:")
print((100 * avg_norm_acc_ll).round(2))


Average normalized accuracy (from LogLoss) - Interpolation:
model
Const.         6.35
Engression    26.54
FT-Trans.     71.88
GPBoost       65.37
GBT            6.63
Log. Regr.    33.22
MLP           61.39
RF             6.63
ResNet        58.57
TabPFN        99.15
dtype: float64


In [26]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_ll = avg_rank(pivot_ll, error_metric=True)
print("Average rank (LogLoss) - Interpolation:")
print(avg_rank_ll.round(2))



Average rank (LogLoss) - Interpolation:
model
Const.        9.00
Engression    6.70
FT-Trans.     3.55
GPBoost       3.95
GBT           7.41
Log. Regr.    6.48
MLP           4.09
RF            7.41
ResNet        4.52
TabPFN        1.35
dtype: float64


In [27]:
avg_diff2    = avg_relative_diff(pivot_ll, error_metric=True)           # in %
avg_acc2     = normalized_accuracy(pivot_ll, error_metric=True) * 100   # convert to % 
avg_rank2    = avg_rank(pivot_ll, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff2, avg_acc2, avg_rank2],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries2 = pd.concat([pivot_ll, metrics])

out2 = pivot_with_summaries2.reset_index().rename(columns={"index":"task_id"})

out2.to_csv(f"avg_{metric2}_with_summary_IP.csv", float_format="%.3f", index=False)

latex2 = out2.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric2}_IP_results.tex","w") as f:
    f.write(latex2)

Average Difference and Average Relative Difference Accuracy

In [29]:
metric3 = "Accuracy"
dfm3   = df[df["metric"] == metric3]

pivot_acc = dfm3.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_acc = pivot_acc.rename(columns=rename_map)
pivot_acc.to_csv(f"avg_{metric3}_per_task_per_model_IP.csv", float_format="%.5f")


In [30]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)     
                         .mul(-1)                         
                         .div(best_per_task , axis=0)
                         * 100)

    return rel_diff.mean(axis=0)


avg_diff_accuracy = avg_relative_diff(pivot_acc, error_metric=False)

print("\nAverage relative difference (Accuracy) - Interpolation:")
print(avg_diff_accuracy.round(2))



Average relative difference (Accuracy) - Interpolation:
model
Const.        43.91
Engression    18.44
FT-Trans.      3.29
GPBoost        6.86
GBT            1.08
Log. Regr.     9.14
MLP            4.11
RF             2.12
ResNet         3.89
TabPFN         0.19
dtype: float64


In [31]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_acc = normalized_accuracy(pivot_acc, error_metric=False)
print("Average normalized accuracy (from Accuracy) - Interpolation:")
print((100 * avg_norm_acc_acc).round(3))


Average normalized accuracy (from Accuracy) - Interpolation:
model
Const.         0.000
Engression     1.240
FT-Trans.     62.371
GPBoost       29.618
GBT           88.401
Log. Regr.    10.083
MLP           50.767
RF            78.002
ResNet        55.907
TabPFN        96.149
dtype: float64


In [32]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_acc = avg_rank(pivot_acc, error_metric=False)
print("Average rank (Accuracy) -Interpolation:")
print(avg_rank_acc.round(2))



Average rank (Accuracy) -Interpolation:
model
Const.        9.91
Engression    8.74
FT-Trans.     4.27
GPBoost       6.41
GBT           2.65
Log. Regr.    7.43
MLP           5.17
RF            3.43
ResNet        4.91
TabPFN        1.65
dtype: float64


In [33]:
avg_diff3    = avg_relative_diff(pivot_acc, error_metric=False)           # in %
avg_acc3     = normalized_accuracy(pivot_acc, error_metric=False) * 100   # convert to % 
avg_rank3    = avg_rank(pivot_acc, error_metric=False)

metrics = pd.DataFrame(
    [avg_diff3, avg_acc3, avg_rank3],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries3 = pd.concat([pivot_acc, metrics])

out3 = pivot_with_summaries3.reset_index().rename(columns={"index":"task_id"})

out3.to_csv(f"avg_{metric3}_with_summary_IP.csv", float_format="%.3f", index=False)

latex3 = out3.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric3}_IP_results.tex","w") as f:
    f.write(latex3)